In [1]:
import pandas as pd
import os
import time
import numpy as np

In [2]:
fpath = os.getcwd()  # assuming df is in current dir
df = pd.read_csv("12Z_soundings.csv")
print(df.head())

   Unnamed: 0  pressure  height  temperature  dewpoint  direction  speed  \
0           0     966.0     379         10.0       3.0      145.0    2.0   
1           1     959.0     440         11.0       3.0      166.0    5.0   
2           2     939.8     610         10.2       1.6      225.0   12.0   
3           3     925.0     743          9.6       0.6      255.0    8.0   
4           4     905.9     914          8.2       0.2      295.0   10.0   

     u_wind    v_wind station  station_number                 time  latitude  \
0 -1.147153  1.638304    CWVK           73033  2018-05-01 12:00:00     50.28   
1 -1.209609  4.851479    CWVK           73033  2018-05-01 12:00:00     50.28   
2  8.485281  8.485281    CWVK           73033  2018-05-01 12:00:00     50.28   
3  7.727407  2.070552    CWVK           73033  2018-05-01 12:00:00     50.28   
4  9.063078 -4.226183    CWVK           73033  2018-05-01 12:00:00     50.28   

   longitude  elevation     pw  ddmmyyyy  
0    -119.27      3

In [3]:
df2 = df[["pw", "ddmmyyyy"]]
df3 = df[["speed", "ddmmyyyy"]]
df4 = df[["pressure", "speed", "ddmmyyyy"]]
df_tke = df[["height", "u_wind", "v_wind", "ddmmyyyy"]]
print(df_tke.head())

   height    u_wind    v_wind  ddmmyyyy
0     379 -1.147153  1.638304   1052018
1     440 -1.209609  4.851479   1052018
2     610  8.485281  8.485281   1052018
3     743  7.727407  2.070552   1052018
4     914  9.063078 -4.226183   1052018


In [7]:
# check the measurements per day
t = time.process_time()

print(df2
 .value_counts()
 .groupby("ddmmyyyy")
 .head(1)
 .reset_index(name="Count")
 )

elapsed_time = time.process_time() - t
print(elapsed_time)

        pw  ddmmyyyy  Count
0    14.79  15052018    157
1    13.67  10052018    150
2    13.29   6062018    146
3    17.43  25052018    146
4    21.98   5072018    146
..     ...       ...    ...
687   9.58  14072018     35
688   0.00   4062020     28
689   0.00  29062022     27
690   0.00   5052020     25
691  13.43  25062018     16

[692 rows x 3 columns]
0.039423032999999386


In [12]:
# wind the strongest mean wind each day
t = time.process_time()

print(df3
 .groupby(["ddmmyyyy"])
 .agg({"speed" : "mean"})
 .rename(columns={"speed": "mean"})
 .sort_values(["mean"], ascending=False)
 ) 

elapsed_time = time.process_time() - t
print(elapsed_time)

               mean
ddmmyyyy           
26092019  62.510204
1072018   53.938272
24092019  53.378947
25092019  50.802083
2062018   48.192771
...             ...
10052022   9.309278
5082018    8.927711
22082018   8.130000
21062018   7.041667
11092019   5.988372

[692 rows x 1 columns]
0.02621321099999996


In [4]:
# wind the strongest mean wind  each day
t = time.process_time()

print(df3
 .groupby(["ddmmyyyy"])
 .agg({"speed" : "mean"})
 .rename(columns={"speed": "mean"})
 .sort_values(["mean"], ascending=False)
 ) 

elapsed_time = time.process_time() - t
print(elapsed_time)

               mean
ddmmyyyy           
26092019  62.510204
1072018   53.938272
24092019  53.378947
25092019  50.802083
2062018   48.192771
...             ...
10052022   9.309278
5082018    8.927711
22082018   8.130000
21062018   7.041667
11092019   5.988372

[692 rows x 1 columns]
0.011078920999999964


In [5]:
# get lowest ~50mb of wind speed and get daily maximums
t = time.process_time()

df4 = df4.loc[df4["pressure"] > 940]
print(df4
      .groupby(["ddmmyyyy"])
      .agg({"speed" : "max"})
      .rename(columns={"speed": "max"})
      .sort_values(["max"], ascending=False)
     )

elapsed_time = time.process_time() - t
print(elapsed_time)

           max
ddmmyyyy      
3092018   21.0
20072018  21.0
8062022   18.0
28052021  17.0
19052018  16.0
...        ...
13052019   0.0
13062019   0.0
31052018   0.0
28062020   0.0
1072020    0.0

[688 rows x 1 columns]
0.006715414999999947


In [9]:
# find the mean tke in the lowest 1500m for each day
t = time.process_time()

df_tke = df_tke.loc[df_tke["height"] < 1500]  # all values wih height less than 
df_tke["u_var"] = np.nan  # primed values 
df_tke["v_var"] = np.nan

dates = df_tke["ddmmyyyy"].unique()

# get the daily variance values
uu = df_tke.groupby(["ddmmyyyy"]).agg({"u_wind" : "var"}).rename(columns={"u_wind" : "u_var"})
uu.set_index(dates)
vv = df_tke.groupby(["ddmmyyyy"]).agg({"v_wind" : "var"}).rename(columns={"v_wind" : "v_var"})
vv.set_index(dates)

# concatinate the two frames to calculate daily tke
frames = [uu, vv]
tke = pd.concat(frames, axis=1)
tke["tke"] = 0.5 * (tke["u_var"]**2 + tke["v_var"]**2)  # get half the square of both variance values

print(tke.sort_values(by=["tke"], ascending=False))

elapsed_time = time.process_time() - t
print(elapsed_time)   

               u_var       v_var          tke
ddmmyyyy                                     
25062018  117.634197   14.944404  7030.569771
15092021   18.866299  110.032897  6231.587796
16062018    3.814501   97.658267  4775.843795
4052022    83.874453   16.435040  3652.517193
7072018    85.196530    5.104336  3642.251526
...              ...         ...          ...
5082019     0.418119    0.889550     0.483061
2072022     0.386071    0.825657     0.415380
9072018     0.370044    0.520352     0.203849
31052022    0.421105    0.388474     0.164120
1072022     0.227515    0.297967     0.070274

[689 rows x 3 columns]
0.0390332330000005


In [3]:
# re-run the same commands with the cudf extension instead (dont have to change any syntax)
get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [1]:
import cudf
%load_ext cudf.pandas

import pandas as pd
import os
import time
import numpy as np

In [2]:
fpath = os.getcwd()  # assuming df is in current dir
df = pd.read_csv("12Z_soundings.csv")
print(df.head())

   Unnamed: 0  pressure  height  temperature  dewpoint  direction  speed  \
0           0     966.0     379         10.0       3.0      145.0    2.0   
1           1     959.0     440         11.0       3.0      166.0    5.0   
2           2     939.8     610         10.2       1.6      225.0   12.0   
3           3     925.0     743          9.6       0.6      255.0    8.0   
4           4     905.9     914          8.2       0.2      295.0   10.0   

     u_wind    v_wind station  station_number                 time  latitude  \
0 -1.147153  1.638304    CWVK           73033  2018-05-01 12:00:00     50.28   
1 -1.209609  4.851479    CWVK           73033  2018-05-01 12:00:00     50.28   
2  8.485281  8.485281    CWVK           73033  2018-05-01 12:00:00     50.28   
3  7.727407  2.070552    CWVK           73033  2018-05-01 12:00:00     50.28   
4  9.063078 -4.226183    CWVK           73033  2018-05-01 12:00:00     50.28   

   longitude  elevation     pw  ddmmyyyy  
0    -119.27      3

In [3]:
df2 = df[["pw", "ddmmyyyy"]]
df3 = df[["speed", "ddmmyyyy"]]
df4 = df[["pressure", "speed", "ddmmyyyy"]]
df_tke = df[["height", "u_wind", "v_wind", "ddmmyyyy"]]
print(df_tke.head())

   height    u_wind    v_wind  ddmmyyyy
0     379 -1.147153  1.638304   1052018
1     440 -1.209609  4.851479   1052018
2     610  8.485281  8.485281   1052018
3     743  7.727407  2.070552   1052018
4     914  9.063078 -4.226183   1052018


In [7]:
# wind the strongest mean wind  each day
t = time.process_time()

print(df3
 .groupby(["ddmmyyyy"])
 .agg({"speed" : "mean"})
 .rename(columns={"speed": "mean"})
 .sort_values(["mean"], ascending=False)
 ) 

elapsed_time = time.process_time() - t
print(elapsed_time)

               mean
ddmmyyyy           
26092019  62.510204
1072018   53.938272
24092019  53.378947
25092019  50.802083
2062018   48.192771
...             ...
10052022   9.309278
5082018    8.927711
22082018   8.130000
21062018   7.041667
11092019   5.988372

[692 rows x 1 columns]
0.022601546999999833


In [10]:
# find the mean tke in the lowest 1500m for each day
t = time.process_time()

df_tke = df_tke.loc[df_tke["height"] < 1500]  # all values wih height less than 
df_tke["u_var"] = np.nan  # primed values 
df_tke["v_var"] = np.nan

dates = df_tke["ddmmyyyy"].unique()

# get the daily variance values
uu = df_tke.groupby(["ddmmyyyy"]).agg({"u_wind" : "var"}).rename(columns={"u_wind" : "u_var"})
uu.set_index(dates)
vv = df_tke.groupby(["ddmmyyyy"]).agg({"v_wind" : "var"}).rename(columns={"v_wind" : "v_var"})
vv.set_index(dates)

# concatinate the two frames to calculate daily tke
frames = [uu, vv]
tke = pd.concat(frames, axis=1)
tke["tke"] = 0.5 * (tke["u_var"]**2 + tke["v_var"]**2)  # get half the square of both variance values

print(tke.sort_values(by=["tke"], ascending=False))

elapsed_time = time.process_time() - t
print(elapsed_time) 

               u_var       v_var          tke
ddmmyyyy                                     
25062018  117.634197   14.944404  7030.569771
15092021   18.866299  110.032897  6231.587796
16062018    3.814501   97.658267  4775.843795
4052022    83.874453   16.435040  3652.517193
7072018    85.196530    5.104336  3642.251526
...              ...         ...          ...
5082019     0.418119    0.889550     0.483061
2072022     0.386071    0.825657     0.415380
9072018     0.370044    0.520352     0.203849
31052022    0.421105    0.388474     0.164120
1072022     0.227515    0.297967     0.070274

[689 rows x 3 columns]
0.039358788000000366


In [12]:
# profiling
new_df = pd.DataFrame({'a': [0, 1, 2], 'b': [100, 101, 102]})
new_df = pd.concat([new_df, new_df], axis=0)

print(new_df)

   a    b
0  0  100
1  1  101
2  2  102
0  0  100
1  1  101
2  2  102


In [13]:
%%cudf.pandas.profile

axis=0
for ii in range(0, 2):
    new_df.max(axis=axis)
    axis=1
    
counts = new_df.groupby("a").b.count()
print(counts)

a
0    2
1    2
2    2
Name: b, dtype: int64


                                                                                                         
                                        Total time elapsed: 2.650 seconds                                
                                      5 GPU function calls in 2.305 seconds                              
                                      0 CPU function calls in 0.000 seconds                              
                                                                                                         
                                                      Stats                                              
                                                                                                         
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function            ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame.max       │ 2          │ 2.252       │ 1.126       │ 0          │ 0.000       │ 0.000       │
│ DataFrame.groupby   │ 1          │ 0.001       │ 0.001       │ 0          │ 0.000       │ 0.000       │
│ SeriesGroupBy.count │ 1          │ 0.040       │ 0.040       │ 0          │ 0.000       │ 0.000       │
│ Series.__str__      │ 1          │ 0.011       │ 0.011       │ 0          │ 0.000       │ 0.000       │
└─────────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [15]:
%%cudf.pandas.line_profile

new_df = pd.DataFrame({'a': [0, 1, 2], 'b': [100, 101, 102]})
new_df = pd.concat([new_df, new_df], axis=0)

axis=0
for ii in range(0, 2):
    new_df.max(axis=axis)
    axis=1
    
counts = new_df.groupby("a").b.count()
print(counts)

a
0    2
1    2
2    2
Name: b, dtype: int64


                                                                                                            
                                         Total time elapsed: 2.470 seconds                                  
                                                                                                            
                                                       Stats                                                
                                                                                                            
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                              ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 2        │     new_df = pd.DataFrame({'a': [0, 1, 2], 'b': [100, 101, 102]}) │ 0.004788732 │             │
│          │                                                                   │             │             │
│ 3        │     new_df = pd.concat([new_df, new_df], axis=0)                  │ 0.011065640 │             │
│          │                                                                   │             │             │
│ 5        │     axis=0                                                        │             │             │
│          │                                                                   │             │             │
│ 6        │     for ii in range(0, 2):                                        │             │             │
│          │                                                                   │             │             │
│ 7        │         new_df.max(axis=axis)                                     │ 0.019843843 │             │
│          │                                                                   │             │             │
│ 8        │         axis=1                                                    │             │             │
│          │                                                                   │             │             │
│ 10       │     counts = new_df.groupby("a").b.count()                        │ 0.013493804 │             │
│          │                                                                   │             │             │
│ 11       │     print(counts)                                                 │ 0.011784832 │             │
│          │                                                                   │             │             │
└──────────┴───────────────────────────────────────────────────────────────────┴─────────────┴─────────────┘

In [18]:
%%cudf.pandas.profile
new_df.count(axis=0)

a    6
b    6
dtype: int64

                                                                                                     
                                      Total time elapsed: 0.269 seconds                              
                                    3 GPU function calls in 0.015 seconds                            
                                    0 CPU function calls in 0.000 seconds                            
                                                                                                     
                                                    Stats                                            
                                                                                                     
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function        ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame.count │ 1          │ 0.004       │ 0.004       │ 0          │ 0.000       │ 0.000       │
│ Series          │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Series.__repr__ │ 1          │ 0.011       │ 0.011       │ 0          │ 0.000       │ 0.000       │
└─────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

In [19]:
%%cudf.pandas.profile
new_df.count(axis=1)

0    2
1    2
2    2
0    2
1    2
2    2
dtype: int64

                                                                                                     
                                      Total time elapsed: 0.277 seconds                              
                                    2 GPU function calls in 0.012 seconds                            
                                    1 CPU function calls in 0.017 seconds                            
                                                                                                     
                                                    Stats                                            
                                                                                                     
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Function        ┃ GPU ncalls ┃ GPU cumtime ┃ GPU percall ┃ CPU ncalls ┃ CPU cumtime ┃ CPU percall ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ DataFrame.count │ 0          │ 0.000       │ 0.000       │ 1          │ 0.017       │ 0.017       │
│ Series          │ 1          │ 0.000       │ 0.000       │ 0          │ 0.000       │ 0.000       │
│ Series.__repr__ │ 1          │ 0.011       │ 0.011       │ 0          │ 0.000       │ 0.000       │
└─────────────────┴────────────┴─────────────┴─────────────┴────────────┴─────────────┴─────────────┘

Not all pandas operations ran on the GPU. The following functions required CPU fallback:

- DataFrame.count

To request GPU support for any of these functions, please file a Github issue here: 
]8;id=511796;https://github.com/rapidsai/cudf/issues/new?assignees=&labels=%3F+-+Needs+Triage%2C+feature+request&projects=&template=pandas_function_request.md&title=%5BFEA%5D\https://github.com/rapidsai/cudf/issues/new/choose]8;;\.